In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import pickle

import numpy
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from tensorflow.examples.tutorials.mnist import input_data

tf.enable_eager_execution()

In [7]:
import tf_func.layers as L
import tf_func.margin_loss as lmargin

In [3]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [4]:
file_name = "/home/xyang2/project/data/dataset/mnist/seed1/labeled_train.tfrecords"

In [5]:
def data_input_fn(filenames, batch_size=1000, shuffle=False):
    
    def _parser(record):
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string)
        }
        parsed_record = tf.parse_single_example(record, features)
        image = tf.decode_raw(parsed_record['image'], tf.float32)

        label = tf.cast(parsed_record['label'], tf.int32)

        return image, tf.one_hot(label, depth=10)
    
    def _iter():
        dataset = (tf.data.TFRecordDataset(filenames)
            .map(_parser))
        if shuffle:
            dataset = dataset.shuffle(buffer_size=10_000)

        dataset = dataset.repeat(None) # Infinite iterations: let experiment determine num_epochs
        dataset = dataset.batch(batch_size)
        
        iterator = dataset.make_one_shot_iterator()
        return iterator
    
    def _input_fn():        
        iterator = _iter()
        features, labels = iterator.get_next()
        
        return features, labels
    return _iter

In [6]:
iterator = data_input_fn("./mnist/train-1.tfrecords", batch_size=100)()
val_iterator = data_input_fn("./mnist/validation.tfrecords", batch_size=100)()
images, labels = iterator.get_next()
images.numpy().sum()

10697.64

In [8]:
def mlp(x, is_training=True, update_batch_stats=True, stochastic=True, seed=1234):
    h = x
    endpoints = {}
    rng = np.random.RandomState(seed)
    # h = tf.layers.flatten(h)
    h = L.fc(h, 784, 1200, seed=rng.randint(123456), name='fc1')
    h = L.relu(L.bn(h, 1200, is_training=is_training, update_batch_stats=update_batch_stats, name='b1'))
    endpoints["fc1"] = h
    h = L.fc(h, 1200, 1200, seed=rng.randint(123456), name='fc2')
    h = L.relu(L.bn(h, 1200, is_training=is_training, update_batch_stats=update_batch_stats, name='b2'))
    endpoints["fc2"] = h
    h = L.fc(h, 1200, 10, seed=rng.randint(123456), name='fc')

    if True:
        h = L.bn(h, 10, is_training=is_training, update_batch_stats=update_batch_stats, name='bfc')

    return h, endpoints

In [40]:
nll_loss

<tf.Tensor: id=7501, shape=(), dtype=float32, numpy=0.14169922>

In [ ]:
optimizer = tf.train.AdamOptimizer(0.001)

In [ ]:
for i in range(50):
    loss, grads_and_vars = cal_gradient(x, y)
    optimizer.apply_gradients(grads_and_vars)
    print("step: {}  loss: {}".format(step, loss.numpy()))

In [11]:
def softmax_model(model, image_batch):
    model_output = tf.nn.softmax(model(image_batch)[0])
    return model_output

data = input_data.read_data_sets("data/MNIST_data/", one_hot=True)
train_ds = tf.data.Dataset.from_tensor_slices((data.train.images, data.train.labels))\
    .map(lambda x, y: (x, tf.cast(y, tf.float32)))\
    .shuffle(buffer_size=1000)\
    .batch(100)\

model = mlp
optimizer = tf.train.AdamOptimizer(0.001)

it = tfe.Iterator(train_ds)
image_batch, label_batch = it.next()


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [12]:
for step in range(100):
    loss, grads_and_vars = cal_gradient(model, image_batch, label_batch)

    optimizer.apply_gradients(grads_and_vars)
    
    print("step: {}  loss: {}".format(step, loss.numpy()))

    if step % 10 == 0:
        model_test_output = softmax_model(model, data.test.images)
        model_test_label = data.test.labels
        correct_prediction = tf.equal(tf.argmax(model_test_output, 1), tf.argmax(model_test_label, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        print("test accuracy = {}".format(accuracy.numpy()))

step: 0  loss: 2.737337350845337
test accuracy = 0.1062999963760376
step: 1  loss: 2.5117690563201904
step: 2  loss: 2.625941514968872
step: 3  loss: 2.5772452354431152
step: 4  loss: 3.005286455154419
step: 5  loss: 2.7645673751831055
step: 6  loss: 2.7314765453338623
step: 7  loss: 2.449230432510376
step: 8  loss: 2.8209028244018555
step: 9  loss: 2.541626214981079
step: 10  loss: 2.708069086074829
test accuracy = 0.0925000011920929
step: 11  loss: 2.415274143218994
step: 12  loss: 2.7800986766815186
step: 13  loss: 2.892584800720215
step: 14  loss: 2.6332077980041504
step: 15  loss: 2.8512630462646484
step: 16  loss: 2.949721336364746
step: 17  loss: 2.599675178527832
step: 18  loss: 2.8251454830169678
step: 19  loss: 2.8685660362243652
step: 20  loss: 2.6079752445220947
test accuracy = 0.09889999777078629
step: 21  loss: 2.708016872406006
step: 22  loss: 2.6244702339172363
step: 23  loss: 2.7577314376831055
step: 24  loss: 2.8216304779052734
step: 25  loss: 2.8594720363616943
step:

In [9]:
def cross_entropy(model_output, label_batch):
    loss = tf.reduce_mean(
        -tf.reduce_sum(label_batch * tf.log(model_output),
        reduction_indices=[1]))
    return loss


@tfe.implicit_value_and_gradients
def cal_gradient(model, image_batch, label_batch):
    return cross_entropy(softmax_model(model, image_batch), label_batch)

In [15]:
model = MLP(None)
logits, endpoints = model(image_batch, is_training=True)


(100, 784)


In [19]:
for layer in layers_list:
    print(layer)
    # break

tf.Tensor(
[[0.18038413 0.22659564 0.         ... 0.         0.         0.3759577 ]
 [0.36179423 0.36371958 0.         ... 0.         0.         0.        ]
 [0.20473713 0.3723094  0.         ... 0.         0.1492198  0.        ]
 ...
 [0.25645143 0.6003772  0.         ... 0.         0.13306507 0.        ]
 [0.01401306 0.30898386 0.         ... 0.         0.         0.        ]
 [0.05506128 0.23868443 0.00085331 ... 0.         0.         0.29849178]], shape=(100, 1200), dtype=float32)
tf.Tensor(
[[0.06070615 0.04257288 0.05172792 ... 0.29537678 0.         0.1104888 ]
 [0.         0.         0.25382587 ... 0.         0.14778559 0.        ]
 [0.         0.04479885 0.07401633 ... 0.         0.         0.        ]
 ...
 [0.16355065 0.         0.         ... 0.26173037 0.         0.        ]
 [0.02450862 0.         0.         ... 0.05797724 0.         0.04237015]
 [0.         0.10344929 0.         ... 0.4595394  0.00108536 0.09780887]], shape=(100, 1200), dtype=float32)
tf.Tensor(
[[0.06070

In [18]:
model = MLP(None)

one_hot_labels = label_batch
top_k = 1
with tf.GradientTape() as t:
    logits, endpoints = model(image_batch, is_training=True)
    layers_list = [endpoints[e] for e in endpoints]
    class_prob = tf.nn.softmax(logits)
    # Pick the correct class probability.
    correct_class_prob = tf.reduce_sum(class_prob * one_hot_labels, axis=1, keepdims=True)

    # Class probabilities except the correct.
    other_class_prob = class_prob * (1. - one_hot_labels)
    if top_k > 1:
        # Pick the top k class probabilities other than the correct.
        top_k_class_prob, _ = tf.nn.top_k(other_class_prob, k=top_k)
    else:
        top_k_class_prob = tf.reduce_max(other_class_prob, axis=1, keepdims=True)

    # Difference between correct class probailities and top_k probabilities.
    difference_prob = correct_class_prob - top_k_class_prob
    losses_list = []
    for layer in layers_list:
        difference_prob_grad = [
            tf.layers.flatten(t.gradient(difference_prob[:, i], layer)[0])
            for i in range(top_k)
        ]

(100, 784)


ValueError: Input 0 of layer flatten_9 is incompatible with the layer: : expected min_ndim=2, found ndim=1. Full shape received: [1200]

In [58]:
large_margin(logits=logits, one_hot_labels=label_batch, layers_list=endpoints)

RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [55]:

def get_norm_fn(norm_type):
    norm_fn = lambda x: tf.norm(x, ord=norm_type)
    return norm_fn


def maximum_with_relu(a, b):
    return a + tf.nn.relu(b - a)


def _ensure_large_margin_args(name, sentinel, one_hot_labels, logits, layers_list, dist_norm, loss_type):
    """Ensures arguments are correct."""
    # Make sure that all arguments were passed as named arguments.
    if sentinel is not None:
        raise ValueError(
            "Only call `%s` with "
            "named arguments (one_hot_labels=..., logits=..., ...)" % name)
    if one_hot_labels is None or logits is None or not layers_list:
        raise ValueError("logits, one_hot_labels and layers_list must be provided.")

    if dist_norm not in {1, 2, np.inf}:
        raise ValueError("dist_norm must be 1, 2, or np.inf.")

    if loss_type not in {"all_top_k", "average_top_k", "worst_top_k"}:
        raise ValueError(
            "loss_type must be 'all_top_k', 'average_top_k', or 'worst_top_k'.")


# pylint: disable=invalid-name
def large_margin(_sentinel=None, logits=None, one_hot_labels=None, layers_list=None, gamma=10000, alpha_factor=2, top_k=1, dist_norm=2,
                 epsilon=1e-8, use_approximation=True, loss_type="all_top_k", loss_collection=tf.GraphKeys.LOSSES):
    """Creates a large margin loss.

    Args:
        _sentinel: Used to prevent positional parameters. Internal, do not use.
        logits: Float `[batch_size, num_classes]` logits outputs of the network.
        one_hot_labels: `[batch_size, num_classes]` Target integer labels in `{0,
            1}`.
        layers_list: List of network Tensors at different layers. The large margin
            is enforced at the layers specified.
        gamma: Desired margin, and distance to boundary above the margin will be
            clipped.
        alpha_factor: Factor to determine the lower bound of margin. Both gamma and
            alpha_factor determine points to include in training the margin these
            points lie with distance to boundary of [gamma * (1 - alpha), gamma]
        top_k: Number of top classes to include in the margin loss.
        dist_norm: Distance to boundary defined on norm (options: be 1, 2, np.inf).
        epsilon: Small number to avoid division by 0.
        use_approximation: If true, use approximation of the margin gradient for
            less computationally expensive training.
        loss_type: 'worst_top_k', 'average_top_k', or 'all_top_k'. If 'worst_top_k'
            only consider the minimum distance to boundary of the top_k classes. If
            'average_top_k' consider average distance to boundary. If 'all_top_k'
            consider all top_k. When top_k = 1, these choices are equivalent.
        loss_collection: Collection to which the loss will be added.

    Returns:
        loss: Scalar `Tensor` of the same type as `logits`.
    Raises:
        ValueError: If the shape of `logits` doesn't match that of
            `one_hot_labels`.    Also if `one_hot_labels` or `logits` is None.
    """

    _ensure_large_margin_args("large_margin", _sentinel, one_hot_labels, logits, layers_list, dist_norm, loss_type)
    logits = tf.convert_to_tensor(logits)
    one_hot_labels = tf.cast(one_hot_labels, logits.dtype)
    logits.get_shape().assert_is_compatible_with(one_hot_labels.get_shape())
    assert top_k > 0
    assert top_k <= logits.get_shape()[1]

    dual_norm = {1: np.inf, 2: 2, np.inf: 1}
    norm_fn = get_norm_fn(dual_norm[dist_norm])
    with tf.name_scope("large_margin_loss"):
        class_prob = tf.nn.softmax(logits)
        # Pick the correct class probability.
        correct_class_prob = tf.reduce_sum(class_prob * one_hot_labels, axis=1, keepdims=True)

        # Class probabilities except the correct.
        other_class_prob = class_prob * (1. - one_hot_labels)
        if top_k > 1:
            # Pick the top k class probabilities other than the correct.
            top_k_class_prob, _ = tf.nn.top_k(other_class_prob, k=top_k)
        else:
            top_k_class_prob = tf.reduce_max(other_class_prob, axis=1, keepdims=True)

        # Difference between correct class probailities and top_k probabilities.
        difference_prob = correct_class_prob - top_k_class_prob
        losses_list = []
        for layer in layers_list:
            difference_prob_grad = [
                tf.layers.flatten(tf.gradients(difference_prob[:, i], layer)[0])
                for i in range(top_k)
            ]

            difference_prob_gradnorm = tf.concat([
                tf.map_fn(norm_fn, difference_prob_grad[i])[:, tf.newaxis]
                for i in range(top_k)
            ], axis=1)

            if use_approximation:
                difference_prob_gradnorm = tf.stop_gradient(difference_prob_gradnorm)

            distance_to_boundary = difference_prob / (difference_prob_gradnorm + epsilon)

            if loss_type == "worst_top_k":
                # Only consider worst distance to boundary.
                distance_to_boundary = tf.reduce_min(distance_to_boundary, axis=1)

            elif loss_type == "average_top_k":
                # Only consider average distance to boundary.
                distance_to_boundary = tf.reduce_mean(distance_to_boundary, axis=1)

            # Distances to consider between distance_upper and distance_lower bounds
            distance_upper = gamma
            distance_lower = gamma * (1 - alpha_factor)

            # Enforce lower bound.
            loss_layer = maximum_with_relu(distance_to_boundary, distance_lower)

            # Enforce upper bound.
            loss_layer = maximum_with_relu(
                0, distance_upper - loss_layer) - distance_upper

            losses_list.append(tf.reduce_mean(loss_layer))

        loss = tf.reduce_mean(losses_list)
        # Add loss to loss_collection.
        tf.losses.add_loss(loss, loss_collection)
    return loss

In [8]:

data = input_data.read_data_sets("/home/xyang2/project/data/dataset/mnist/", one_hot=True)
train_ds = tf.data.Dataset.from_tensor_slices((data.train.images, data.train.labels))\
    .map(lambda x, y: (x, tf.cast(y, tf.float32)))\
    .shuffle(buffer_size=1000)\
    .batch(100)\

optimizer = tf.train.GradientDescentOptimizer(0.5)

for step, (image_batch, label_batch) in enumerate(tfe.Iterator(train_ds)):
    print(image_batch.shape)
    loss, grads_and_vars = cal_gradient(image_batch, label_batch)
    optimizer.apply_gradients(grads_and_vars)
    if step % 100:
        print("step: {}  loss: {}".format(step, loss.numpy()))
    break

model_test_output = softmax_model(data.test.images)
model_test_label = data.test.labels
correct_prediction = tf.equal(tf.argmax(model_test_output, 1), tf.argmax(model_test_label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("test accuracy = {}".format(accuracy.numpy()))


Extracting /home/xyang2/project/data/dataset/mnist/train-images-idx3-ubyte.gz
Extracting /home/xyang2/project/data/dataset/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/xyang2/project/data/dataset/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/xyang2/project/data/dataset/mnist/t10k-labels-idx1-ubyte.gz
(100, 784)
(100, 784)
(10000, 784)


AttributeError: 'tuple' object has no attribute 'ndims'